In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00


## Import Libraries and Load Data

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
# import streamlit as st

In [ ]:
import pandas as pd

# I have stored DPM2009 document on drive and uploded link here
csv_link = "https://drive.google.com/uc?id=1f95OsR1duzawT-8sCyyQSsLbOEYALmt7"

# Read the CSV file into a DataFrame
data = pd.read_csv(csv_link)

print(data.head())


   Unnamed: 0                                            context  \
0           0  CHAPTER 52 \n\nNicholas despairs of rescuing M...   
1           1  Islamabad, Pakistan (CNN) -- A massive car bom...   
2           2  WASHINGTON (CNN) -- There are no immediate pla...   
3           3  CHAPTER X \n\nMISTRESS AND MEN \n\nHalf-an-hou...   
4           4  (CNN) -- California's parole board Tuesday ref...   

                             question  \
0             What would happen then?   
1  About how many total were injured?   
2               Where is the meeting?   
3      Were people amazed about this?   
4                   Who's her lawyer?   

                                              answer  
0                         a disagreeable predicament  
1  A massive car bomb tore through the heart of a...  
2    President Obama meet in Washington on Wednesday  
3                         udible breath of amazement  
4                        And lawyer Brandie Devall,   


In [ ]:
# data = pd.read_csv("coqatrain.csv", skiprows=[19962])

In [ ]:
data.head()

,Unnamed: 0,context,question,answer
0,0,CHAPTER 52 \n\nNicholas despairs of rescuing M...,What would happen then?,a disagreeable predicament
1,1,"Islamabad, Pakistan (CNN) -- A massive car bom...",About how many total were injured?,A massive car bomb tore through the heart of a...
2,2,WASHINGTON (CNN) -- There are no immediate pla...,Where is the meeting?,President Obama meet in Washington on Wednesday
3,3,CHAPTER X \n\nMISTRESS AND MEN \n\nHalf-an-hou...,Were people amazed about this?,udible breath of amazement
4,4,(CNN) -- California's parole board Tuesday ref...,Who's her lawyer?,"And lawyer Brandie Devall,"


In [ ]:
print("Number of question and answers: ", len(data))

Number of question and answers:  30000


## Load Pretrained Model and Tokenizer

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
random_num = np.random.randint(0,len(data))
question = data["question"][random_num]
text = data["context"][random_num]


In [ ]:
input_ids = tokenizer.encode(question, text)
print("The input has a total of {} tokens.".format(len(input_ids)))

The input has a total of 351 tokens.


In [ ]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
for token, id in zip(tokens, input_ids):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
how        2,129
did        2,106
this       2,023
make       2,191
her        2,014
feel       2,514
?          1,029
[SEP]        102
chapter    3,127
xiii      15,031
.          1,012
waking    12,447
up         2,039
.          1,012
every      2,296
thing      2,518
did        2,106
"          1,000
go         2,175
beautifully  17,950
"          1,000
for        2,005
a          1,037
time       2,051
;          1,025
so         2,061
much       2,172
so         2,061
,          1,010
that       2,008
christie  13,144
began      2,211
to         2,000
think      2,228
she        2,016
really     2,428
had        2,018
"          1,000
got        2,288
religion   4,676
.          1,012
"          1,000
a          1,037
delightful  26,380
peace      3,521
per        2,566
##va       3,567
##ded      5,732
her        2,014
soul       3,969
,          1,010
a          1,037
new        2,047
interest   3,037
made       2,081
the        1,996
dull      10,634
##est    

In [ ]:
#first occurence of [SEP] token
sep_idx = input_ids.index(tokenizer.sep_token_id)
print("SEP token index: ", sep_idx)
#number of tokens in segment A (question) - this will be one more than the sep_idx as the index in Python starts from 0
num_seg_a = sep_idx+1
print("Number of tokens in segment A: ", num_seg_a)
#number of tokens in segment B (text)
num_seg_b = len(input_ids) - num_seg_a
print("Number of tokens in segment B: ", num_seg_b)
#creating the segment ids
segment_ids = [0]*num_seg_a + [1]*num_seg_b
#making sure that every input token has a segment id
assert len(segment_ids) == len(input_ids)

SEP token index:  8
Number of tokens in segment A:  9
Number of tokens in segment B:  342


In [ ]:
#token input_ids to represent the input and token segment_ids to differentiate our segments - question and text
output = model(torch.tensor([input_ids]),
               token_type_ids=torch.tensor([segment_ids]))

### Test on random Question

In [ ]:
#tokens with highest start and end scores
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits)
if answer_end >= answer_start:
    answer = " ".join(tokens[answer_start:answer_end+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")

print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))


Question:
How did this make her feel?

Answer:
She felt as if she could forgive all her enemies.


In [ ]:
## deal with ##
answer = tokens[answer_start]
for i in range(answer_start+1, answer_end+1):
    if tokens[i][0:2] == "##":
        answer += tokens[i][2:]
    else:
        answer += " " + tokens[i]

In [ ]:
def question_answer(question, text):

    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)

    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a

    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)

    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]

    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."

    print("\nPredicted answer:\n{}".format(answer.capitalize()))

In [ ]:
text = """New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's famous rhinestone-studded glove from a 1983 performance -- were auctioned off Saturday, reaping a total $2 million. Profits from the auction at the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of only $120,000 in sales. The highly prized memorabilia, which included items spanning the many stages of Jackson's career, came from more than 30 fans, associates and family members, who contacted Julien's Auctions to sell their gifts and mementos of the singer. Jackson's flashy glove was the big-ticket item of the night, fetching $420,000 from a buyer in Hong Kong, China. Jackson wore the glove at a 1983 performance during \"Motown 25,\" an NBC special where he debuted his revolutionary moonwalk. Fellow Motown star Walter \"Clyde\" Orange of the Commodores, who also performed in the special 26 years ago, said he asked for Jackson's autograph at the time, but Jackson gave him the glove instead. "The legacy that [Jackson] left behind is bigger than life for me,\" Orange said. \"I hope that through that glove people can see what he was trying to say in his music and what he said in his music.\" Orange said he plans to give a portion of the proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 percent buyer's premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 20 percent premium."""
question = "Where was the Auction held?"
question_answer(question, text)
#original answer from the dataset
print("Original answer:\n", data.loc[data["question"] == question]["answer"])


Predicted answer:
Hard rock cafe in new york ' s times square
Original answer:
 25386    Hard Rock Cafe in New York's Times Square
Name: answer, dtype: object


## save trained model

In [ ]:
# # Save the trained model and tokenizer
# model.save_pretrained("saved_model")
# tokenizer.save_pretrained("saved_model")

# print("Trained model and tokenizer have been saved.")

In [ ]:
# from transformers import BertForQuestionAnswering, BertTokenizer

# # Load the saved model and tokenizer
# model = BertForQuestionAnswering.from_pretrained("saved_model")
# tokenizer = BertTokenizer.from_pretrained("saved_model")

# def question_answer(question, text):
#     # Tokenize question and text as a pair
#     input_ids = tokenizer.encode(question, text)
#     tokens = tokenizer.convert_ids_to_tokens(input_ids)

#     # Segment IDs
#     sep_idx = input_ids.index(tokenizer.sep_token_id)
#     num_seg_a = sep_idx + 1
#     num_seg_b = len(input_ids) - num_seg_a
#     segment_ids = [0] * num_seg_a + [1] * num_seg_b
#     assert len(segment_ids) == len(input_ids)

#     # Model output using input_ids and segment_ids
#     output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))

#     # Reconstructing the answer
#     answer_start = torch.argmax(output.start_logits)
#     answer_end = torch.argmax(output.end_logits)
#     if answer_end >= answer_start:
#         answer = tokens[answer_start]
#         for i in range(answer_start + 1, answer_end + 1):
#             if tokens[i][0:2] == "##":
#                 answer += tokens[i][2:]
#             else:
#                 answer += " " + tokens[i]

#     if answer.startswith("[CLS]"):
#         answer = "Unable to find the answer to your question."

#     print("\nPredicted answer:\n{}".format(answer.capitalize()))

# # User input for context
# text = input("Please enter the context: \n")

# while True:
#     # Get question input from the user
#     question = input("\nEnter a question (or 'exit' to quit): ")

#     if question.lower() == 'exit':
#         break

#     question_answer(question, text)


In [ ]:
# from google.colab import files

# files.download('/content/saved_model/pytorch_model.bin')


## Model testing on custom passage and questions

In [ ]:
text = input("Please enter your text: \n")
#question = input("\nPlease enter your question: \n")

while True:
     # Get question input from the user
    question = input("\nEnter a question (or 'exit' to quit): ")

    if question.lower() == 'exit':
        break

    question_answer(question, text)




Please enter your text: 
This Manual may be called the Defence Procurement Manual, 2009 (DPM 2009, for short). This Manual contains principles and procedure relating to procurement of  goods and services for the Defence Services, Organizations and Establishments, laid down in terms of Rule 135 of the General Financial Rules, 2005 and shall come into force with effect from 1st of April 2009. All on-going cases of procurement in which Request for Proposal (RFP) has already been issued may continue to be regulated by the provisions of the Defence Procurement Manual, 2006.

Enter a question (or 'exit' to quit): what is dpm2009?

Predicted answer:
Defence procurement manual , 2009

Enter a question (or 'exit' to quit): when dpm 2009 is implemented?

Predicted answer:
1st of april 2009

Enter a question (or 'exit' to quit): dpm 2009 is based on?

Predicted answer:
Rule 135 of the general financial rules , 2005

Enter a question (or 'exit' to quit): what is purpose of dpm2009?

Predicted answ

In [ ]:
text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")
while True:
    question_answer(question, text)

    flag = True
    flag_N = False

    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "n":
            print("\nBye!")
            flag = False
            flag_N = True

    if flag_N == True:
        break

Please enter your text: 
This Manual may be called the Defence Procurement Manual, 2009 (DPM 2009, for short). This Manual contains principles and procedure relating to procurement of  goods and services for the Defence Services, Organizations and Establishments, laid down in terms of Rule 135 of the General Financial Rules, 2005 and shall come into force with effect from 1st of April 2009. All on-going cases of procurement in which Request for Proposal (RFP) has already been issued may continue to be regulated by the provisions of the Defence Procurement Manual, 2006.

Please enter your question: 
what is dpm2009?

Predicted answer:
Defence procurement manual , 2009

Do you want to ask another question based on this text (Y/N)? y

Please enter your question: 
what is purpose of dpm2009?

Predicted answer:
Procurement of goods and services for the defence services , organizations and establishments

Do you want to ask another question based on this text (Y/N)? y

Please enter your ques